# Live Map Viewer for Person Walkers

This notebook subscribes to MQTT and displays all walking persons on a live map.
Each person appears with their own unique color.

**Usage:**
1. Run all cells to start the map viewer
2. Launch one or more `person_walker.ipynb` notebooks with different names
3. Watch as persons appear and move on the map in real-time!

In [11]:
# Imports used in this notebook
import asyncio
import json
from typing import Optional

from simulated_city.config import load_config
from simulated_city.mqtt import MqttConnector
from simulated_city.maplibre_live import LiveMapLibreMap

# Define map variable up front so static analysis knows it exists
m: Optional[LiveMapLibreMap] = None

print("✓ Imports OK")

✓ Imports OK


In [6]:
# Add 3D buildings after the map exists
map_obj = globals().get("m")
if map_obj is not None:
    map_obj.add_3d_buildings()
    print("✓ Added 3D buildings")
else:
    print("Run the map creation cell first, then run this cell.")

✓ Added 3D buildings


In [12]:
# City Hall coordinates (Copenhagen)
CITY_HALL_LNGLAT = (12.5683, 55.6761)

# Create and display the map
m = LiveMapLibreMap(center=CITY_HALL_LNGLAT, zoom=16.5, height="700px")
display(m)
print("✓ Map initialized")

✓ Map initialized


In [7]:
# Connect to MQTT broker
cfg = load_config()
connector = MqttConnector(cfg.mqtt, client_id_suffix="map-viewer")
connector.connect()
if not connector.wait_for_connection(timeout=10.0):
    raise RuntimeError("Failed to connect to MQTT broker")

print("✓ Connected to MQTT broker")

✓ Connected to MQTT broker


Disconnected from MQTT broker (reason=Unspecified error). Reconnecting...
Disconnected from MQTT broker (reason=Unspecified error). Reconnecting...
Disconnected from MQTT broker (reason=Unspecified error). Reconnecting...


In [ ]:
# Track persons we've seen
persons_seen = set()

def on_person_location(client, userdata, message):
    """
    Callback when a person location message arrives.
    
    Expected message format:
    {
        "lng": float,
        "lat": float,
        "color": str,
        "name": str,
        "timestamp": float
    }
    """
    try:
        data = json.loads(message.payload.decode())
        name = data["name"]
        lng = data["lng"]
        lat = data["lat"]
        color = data["color"]
        
        # Create marker if this is the first time we see this person
        if name not in persons_seen:
            persons_seen.add(name)
            print(f"  New person on map: {name} (color: {color})")
        
        # Update marker position with color and show name in popup
        marker_id = f"person-{name}"
        m.move_marker(marker_id, (lng, lat), color=color, popup=name)
    
    except Exception as e:
        print(f"Error processing message: {e}")

# Subscribe to all person location updates (wildcard +)
connector.client.on_message = on_person_location
connector.client.subscribe("persons/+/location", qos=0)

print("✓ Subscribed to persons/+/location")
print("Waiting for person location updates...\n")

✓ Subscribed to persons/+/location
Waiting for person location updates...



Disconnected from MQTT broker (reason=Unspecified error). Reconnecting...


In [ ]:
# Keep the notebook running and display status updates
async def status_updater():
    """Periodically show how many persons are being tracked."""
    while True:
        await asyncio.sleep(10)
        if persons_seen:
            person_list = ", ".join(sorted(persons_seen))
            print(f"  Tracking {len(persons_seen)} person(s): {person_list}")
        else:
            print("  No persons detected yet. Start a person_walker.ipynb!")

status_task = asyncio.create_task(status_updater())
print("✓ Map viewer is running!")
print("Run the next cell to stop.")

✓ Map viewer is running!
Run the next cell to stop.


Disconnected from MQTT broker (reason=Unspecified error). Reconnecting...
Disconnected from MQTT broker (reason=Unspecified error). Reconnecting...
Disconnected from MQTT broker (reason=Unspecified error). Reconnecting...
Disconnected from MQTT broker (reason=Unspecified error). Reconnecting...


  No persons detected yet. Start a person_walker.ipynb!
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 3 person(s): Alice, Bob, Charlie
  Tracking 6 person(

In [ ]:
# Stop the viewer
if "status_task" in globals() and status_task is not None and not status_task.done():
    status_task.cancel()

if "demo_walk_task" in globals() and demo_walk_task is not None and not demo_walk_task.done():
    demo_walk_task.cancel()

if "connector" in globals() and connector is not None:
    connector.disconnect()

print("✓ Map viewer stopped.")

✓ Map viewer stopped.


In [32]:
# Clear all markers now (one-click hard reset)
if "m" not in globals() or m is None:
    print("Run Cell 4 (map creation) first.")
else:
    # Stop background tasks that can keep updating markers
    if "status_task" in globals() and status_task is not None and not status_task.done():
        status_task.cancel()
    if "demo_walk_task" in globals() and demo_walk_task is not None and not demo_walk_task.done():
        demo_walk_task.cancel()

    # Pause live MQTT updates so cleared markers do not reappear immediately
    if "connector" in globals() and connector is not None:
        try:
            connector.client.unsubscribe("persons/+/location")
        except Exception:
            pass
        connector.client.on_message = None

    # Remove known marker IDs from live runs + demo runs + legacy examples
    known_demo_names = {"Mourinho", "Mbappe", "Haaland", "Messi", "Ronaldo", "Bellingham"}
    cleanup_names = set(globals().get("persons_seen", set()))
    cleanup_names.update(globals().get("names", []))
    cleanup_names.update(known_demo_names)
    cleanup_names.update({"Alice", "Bob", "Charlie"})

    removed = 0
    for person_name in cleanup_names:
        for marker_id in (f"person-{person_name}", person_name):
            try:
                m.remove_marker(marker_id)
                removed += 1
            except Exception:
                pass

    persons_seen = set()
    print(f"✓ Hard reset complete: removed {removed} marker(s)")
    print("✓ MQTT marker updates paused")
    print("Run Cell 10 to start demo walkers again.")

✓ Hard reset complete: removed 18 marker(s)
✓ MQTT marker updates paused
Run Cell 10 to start demo walkers again.


In [ ]:
# Demo walking pinpoints (independent walkers on a broader circle)
import asyncio
import math
from urllib.parse import quote_plus

if "m" not in globals() or m is None:
    print("Run the map creation cell first.")
else:
    # Stop previous demo walk loop if it exists
    if "demo_walk_task" in globals() and demo_walk_task is not None and not demo_walk_task.done():
        demo_walk_task.cancel()

    # Pause MQTT updates while demo mode runs, so old runs do not re-add markers
    if "connector" in globals() and connector is not None:
        try:
            connector.client.unsubscribe("persons/+/location")
        except Exception:
            pass
        connector.client.on_message = None

    demo_people = {
        "Mourinho": {
            "color": "#e63946",
            "photo": "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Jose_Mourinho_2021.jpg/120px-Jose_Mourinho_2021.jpg",
        },
        "Mbappe": {
            "color": "#457b9d",
            "photo": "https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Kylian_Mbapp%C3%A9_2019.jpg/120px-Kylian_Mbapp%C3%A9_2019.jpg",
        },
        "Haaland": {
            "color": "#2a9d8f",
            "photo": "https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/Erling_Haaland_2023.jpg/120px-Erling_Haaland_2023.jpg",
        },
        "Messi": {
            "color": "#f4a261",
            "photo": "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Lionel_Messi_20180626.jpg/120px-Lionel_Messi_20180626.jpg",
        },
        "Ronaldo": {
            "color": "#9b5de5",
            "photo": "https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Cristiano_Ronaldo_2018.jpg/120px-Cristiano_Ronaldo_2018.jpg",
        },
        "Bellingham": {
            "color": "#ff006e",
            "photo": "https://upload.wikimedia.org/wikipedia/commons/thumb/8/89/Jude_Bellingham_2023.jpg/120px-Jude_Bellingham_2023.jpg",
        },
    }

    center_lng, center_lat = 12.5683, 55.6761
    names = list(demo_people.keys())

    # Remove old markers from previous runs (both current and legacy IDs)
    cleanup_names = set(names)
    cleanup_names.update(globals().get("persons_seen", set()))
    cleanup_names.update({"Alice", "Bob", "Charlie"})

    removed_count = 0
    for old_name in cleanup_names:
        for marker_id in (f"person-{old_name}", old_name):
            try:
                m.remove_marker(marker_id)
                removed_count += 1
            except Exception:
                pass

    # Reset seen-person state so only current demo walkers are tracked
    persons_seen = set()

    # Independent walkers: each has its own orbit on a broader circle
    base_radius = 0.00190
    walkers = {}
    for idx, name in enumerate(names):
        walkers[name] = {
            "angle": idx * (2 * math.pi / len(names)),
            "radius": base_radius + idx * 0.00010,
            "speed": 0.030 + idx * 0.004,
        }

    def position(state):
        lng = center_lng + state["radius"] * math.cos(state["angle"])
        lat = center_lat + state["radius"] * math.sin(state["angle"])
        return lng, lat

    def card_html(name, photo_url, size=72, quote=None, show_close=False):
        if quote is not None:
            quote_js = quote.replace("\\", "\\\\").replace("'", "\\'")
            celebrate_js = (
                "onload=\"try{"
                "window.__quotePlayed=window.__quotePlayed||{};"
                f"if(!window.__quotePlayed['{name}']){{"
                f"window.__quotePlayed['{name}']=true;"
                f"const u=new SpeechSynthesisUtterance('{quote_js}');"
                "u.rate=0.9;u.pitch=0.7;"
                "window.speechSynthesis.cancel();"
                "window.speechSynthesis.speak(u);"
                "}"
                "}catch(e){}\""
            )
        else:
            celebrate_js = ""

        close_button_html = ""
        if show_close:
            close_button_html = (
                "<button "
                "style='position:absolute;top:6px;right:6px;border:none;background:#dc2626;color:#fff;"
                "width:28px;height:28px;border-radius:999px;font-size:18px;font-weight:700;cursor:pointer;line-height:28px;"
                "box-shadow:0 2px 8px rgba(0,0,0,0.35);' "
                "title='Close' "
                "onclick=\"try{const p=this.closest('.maplibregl-popup');if(p){const b=p.querySelector('.maplibregl-popup-close-button');if(b){b.click();}}}catch(e){}\""
                ">×</button>"
            )

        return (
            "<div style='position:relative;text-align:center;min-width:150px;padding:8px 10px 10px 10px;"
            "background:#ffffff;border:1px solid #d1d5db;border-radius:12px;box-shadow:0 6px 18px rgba(0,0,0,0.2);'>"
            f"{close_button_html}"
            f"<img src='{photo_url}' style='width:{size}px;height:{size}px;border-radius:50%;object-fit:cover;border:3px solid #fff;"
            "box-shadow:0 2px 8px rgba(0,0,0,0.25);' "
            f"{celebrate_js}/>"
            f"<div style='margin-top:8px;font-weight:700;font-size:15px;color:#111827;letter-spacing:0.2px'>{name}</div>"
            "</div>"
        )

    def get_quote(name):
        if name == "Ronaldo":
            return "Suiiiiiii!"
        if name == "Mourinho":
            return "I prefer not speak"
        if name == "Mbappe":
            return "Gini Wijnaldum, this is football."
        return None

    # Add initial markers
    for name in names:
        lng, lat = position(walkers[name])
        marker_id = f"person-{name}"

        try:
            m.remove_marker(marker_id)
        except Exception:
            pass

        # No tooltip: details are shown only on click (popup)
        m.add_marker(
            lng,
            lat,
            name=marker_id,
            color=demo_people[name]["color"],
            popup=card_html(
                name,
                demo_people[name]["photo"],
                size=96,
                quote=get_quote(name),
                show_close=True,
            ),
        )
        persons_seen.add(name)

    async def demo_walk_loop():
        while True:
            await asyncio.sleep(0.35)
            for name in names:
                walkers[name]["angle"] += walkers[name]["speed"]
                lng, lat = position(walkers[name])
                marker_id = f"person-{name}"
                m.move_marker(
                    marker_id,
                    (lng, lat),
                    color=demo_people[name]["color"],
                    popup=card_html(
                        name,
                        demo_people[name]["photo"],
                        size=96,
                        quote=get_quote(name),
                        show_close=True,
                    ),
                )

    demo_walk_task = asyncio.create_task(demo_walk_loop())
    print(f"✓ Cleared old markers from previous runs ({removed_count} removed attempts)")
    print("✓ 6 demo walkers are moving one-by-one on a much broader circle")
    print("✓ Details/photos now show only when clicking a pinpoint")
    print("✓ Each popup has a bigger close (×) button")

✓ Cleared old markers from previous runs (18 removed attempts)
✓ 6 demo walkers are moving one-by-one on a much broader circle
✓ Details/photos now show only when clicking a pinpoint
✓ Each popup has a bigger close (×) button
